In [2]:
# pyspark packages
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType

#other needed packages
import re
import os

# Set JAVA_HOME for PySpark
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17'

spark = SparkSession.builder \
    .appName("stock market preds") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/18 02:59:44 WARN Utils: Your hostname, Jeffreys-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.17 instead (on interface en0)
26/01/18 02:59:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/18 02:59:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/18 02:59:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/18 02:59:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# functions to import data, run SQL from file and save back to file
# function to import and clean columns
def import_csv_to_table(table_name, file, format_cols):

    #read source files
    df = spark.read.csv(file, header=True, quote="\"",
                        escape="\"", multiLine=True, inferSchema=True)

    #clean column names
    if format_cols:
        cols_formatted = [re.sub(r"[^a-zA-Z0-9\s]", "", col_name).lower().replace(" ", "_") for col_name in df.columns]
        df = df.toDF(*cols_formatted)

    # create SQL view
    df.createOrReplaceTempView(f"{table_name}")
    return df

#run a SQL step
def sql_step(file):
    with open(file, 'r', encoding='utf-8') as file:
        sql_text = file.read()
    results = spark.sql(sql_text)
    return results

#run SQL and view output inline
def run_sql(file, rowstoshow, print_sql):
    with open(file, 'r', encoding='utf-8') as file:
        sql_text = file.read()
    results = spark.sql(sql_text)
    if print_sql == True: print(sql_text)
    results.show(rowstoshow, truncate=False)

# export data frame to csv
def export_csv(df, output_dir, final_file_name):
    df.coalesce(1).write.csv(output_dir, header=True, mode="overwrite")
    for file in os.listdir(output_dir):
        if file.startswith("part-") and file.endswith(".csv"):
            part_file_path = os.path.join(output_dir, file)
            break
    if part_file_path:
        os.rename(part_file_path, os.path.join(output_dir, final_file_name))
        print(f"CSV saved as: {final_file_name}")
    else:
        print("Error: Part file not found.")

In [9]:
news = import_csv_to_table("news", "raw_data/news_data.csv", False)
stocks = import_csv_to_table("stocks", "raw_data/stock_data.csv", False)

In [10]:
feature_set = sql_step("sql/sentiment_data_prep_gemini.sql")
feature_set.show(30, truncate=False)

+---------------+------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|news_article_id|symbol|news_date |article_text                                                                                                                                                                                                                                                                                                                                                            |percent_daily_price_change|
+---------------+------+----------+---------------------------------------------------------------------------------------------------------------------

In [14]:
# Setup Gemini API
import google.generativeai as genai
import json

# Load API key from file
with open('api_keys.json', 'r') as f:
    api_keys = json.load(f)
API_KEY = api_keys['GEMINI_API_KEY']

genai.configure(api_key=API_KEY)

# List available models to find the correct one
print("Available Gemini models:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(f"  - {m.name}")

# Use gemini-2.0-flash (available in the free tier)
model = genai.GenerativeModel('models/gemini-2.0-flash')
        
print(f"\nUsing model: {model.model_name}")
print("Gemini API configured successfully")

Available Gemini models:
  - models/gemini-2.5-flash
  - models/gemini-2.5-pro
  - models/gemini-2.0-flash-exp
  - models/gemini-2.0-flash
  - models/gemini-2.0-flash-001
  - models/gemini-2.0-flash-exp-image-generation
  - models/gemini-2.0-flash-lite-001
  - models/gemini-2.0-flash-lite
  - models/gemini-2.0-flash-lite-preview-02-05
  - models/gemini-2.0-flash-lite-preview
  - models/gemini-exp-1206
  - models/gemini-2.5-flash-preview-tts
  - models/gemini-2.5-pro-preview-tts
  - models/gemma-3-1b-it
  - models/gemma-3-4b-it
  - models/gemma-3-12b-it
  - models/gemma-3-27b-it
  - models/gemma-3n-e4b-it
  - models/gemma-3n-e2b-it
  - models/gemini-flash-latest
  - models/gemini-flash-lite-latest
  - models/gemini-pro-latest
  - models/gemini-2.5-flash-lite
  - models/gemini-2.5-flash-image
  - models/gemini-2.5-flash-preview-09-2025
  - models/gemini-2.5-flash-lite-preview-09-2025
  - models/gemini-3-pro-preview
  - models/gemini-3-flash-preview
  - models/gemini-3-pro-image-preview
 

In [15]:
# Score sentiment with Gemini
import pandas as pd
from tqdm import tqdm
import time

def score_sentiment_gemini(article_text, max_retries=3):
    """Score sentiment using Gemini API with retry logic."""
    prompt = f"""Analyze this financial news article for its likely impact on stock price.

Article: {article_text[:1000]}  # Limit text length for API

Respond with ONLY a valid JSON object (no markdown, no explanation):
{{"sentiment": "positive" or "negative" or "neutral", "confidence": 0.0 to 1.0}}

Rules:
- "positive" = news likely to increase stock price (upgrades, beats, positive outlook)
- "negative" = news likely to decrease stock price (downgrades, misses, negative outlook)  
- "neutral" = news unlikely to move stock price significantly
- confidence should reflect how certain you are (0.5 = uncertain, 1.0 = very confident)"""
    
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            # Clean response and parse JSON
            text = response.text.strip()
            # Remove markdown code blocks if present
            if text.startswith('```'):
                text = text.split('```')[1]
                if text.startswith('json'):
                    text = text[4:]
            result = json.loads(text.strip())
            return result['sentiment'], result['confidence']
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                print(f"Error scoring: {e}")
                return 'neutral', 0.5  # Default on failure

# Convert to pandas
df = feature_set.toPandas()
print(f"Total articles to score: {len(df)}")

# Score each article with progress bar
sentiments, confidences = [], []
for text in tqdm(df['article_text'], desc="Scoring sentiment with Gemini"):
    sentiment, confidence = score_sentiment_gemini(text)
    sentiments.append(sentiment)
    confidences.append(confidence)
    time.sleep(0.1)  # Rate limiting to stay within free tier (15 RPM = 4 sec between requests, but 0.1 is usually fine)

df['sentiment'], df['confidence'] = sentiments, confidences
print(f"\nSentiment distribution:\n{df['sentiment'].value_counts()}\n")
df[['symbol', 'news_date', 'sentiment', 'confidence', 'percent_daily_price_change']].head(10)

Total articles to score: 35430


Scoring sentiment with Gemini:   0%|          | 1/35430 [00:03<34:10:08,  3.47s/it]

Error scoring: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
Please retry in 51.846261094s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_d

Scoring sentiment with Gemini:   0%|          | 2/35430 [00:06<33:12:22,  3.37s/it]

Error scoring: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
Please retry in 48.543898872s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTie

...

Error scoring: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
Please retry in 55.592709533s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_d

Scoring sentiment with Gemini:   0%|          | 37/35430 [02:03<32:39:47,  3.32s/it]

Error scoring: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
Please retry in 52.319677436s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTie

Scoring sentiment with Gemini:   0%|          | 37/35430 [02:04<33:03:08,  3.36s/it]


KeyboardInterrupt: 

In [ ]:
os.makedirs("processed_data", exist_ok=True)
df.to_csv("processed_data/gemini_news_classifications.csv", index=False)
print(f"Exported {len(df)} rows to processed_data/gemini_news_classifications.csv")

In [ ]:
# Sentiment vs Price Change Analysis
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Box plot: Price change by sentiment
axes[0,0].boxplot([df[df['sentiment']==s]['percent_daily_price_change']*100 for s in ['positive','neutral','negative']], 
                   labels=['Positive','Neutral','Negative'])
axes[0,0].set_ylabel('Daily Price Change (%)')
axes[0,0].set_title('Price Change Distribution by Sentiment (Gemini)')
axes[0,0].axhline(0, color='gray', linestyle='--', alpha=0.5)

# 2. Mean price change by sentiment with error bars
means = df.groupby('sentiment')['percent_daily_price_change'].agg(['mean','std','count'])
colors = {'positive':'green', 'neutral':'gray', 'negative':'red'}
bars = axes[0,1].bar(means.index, means['mean']*100, yerr=means['std']*100/np.sqrt(means['count']), 
                      color=[colors[s] for s in means.index], capsize=5)
axes[0,1].set_ylabel('Mean Daily Price Change (%)')
axes[0,1].set_title('Average Price Change by Sentiment (±SE)')
axes[0,1].axhline(0, color='black', linestyle='-', alpha=0.3)

# 3. Actual direction accuracy
df['actual_direction'] = df['percent_daily_price_change'].apply(lambda x: 'positive' if x > 0 else 'negative')
df['correct'] = ((df['sentiment']=='positive') & (df['actual_direction']=='positive')) | \
                ((df['sentiment']=='negative') & (df['actual_direction']=='negative'))
acc_by_sent = df[df['sentiment']!='neutral'].groupby('sentiment')['correct'].mean()
axes[1,0].bar(acc_by_sent.index, acc_by_sent*100, color=[colors[s] for s in acc_by_sent.index])
axes[1,0].set_ylabel('Accuracy (%)')
axes[1,0].set_title('Direction Prediction Accuracy (excl. neutral)')
axes[1,0].axhline(50, color='red', linestyle='--', label='Random baseline')
axes[1,0].legend()

# 4. Confidence vs accuracy scatter
df['abs_change'] = df['percent_daily_price_change'].abs()
conf_bins = pd.cut(df['confidence'], bins=5)
conf_acc = df[df['sentiment']!='neutral'].groupby(conf_bins, observed=True)['correct'].mean()
axes[1,1].bar(range(len(conf_acc)), conf_acc*100, tick_label=[f'{i.left:.2f}-{i.right:.2f}' for i in conf_acc.index])
axes[1,1].set_xlabel('Confidence Score Range')
axes[1,1].set_ylabel('Accuracy (%)')
axes[1,1].set_title('Prediction Accuracy by Confidence Level')
axes[1,1].axhline(50, color='red', linestyle='--')

plt.tight_layout()
plt.show()

# Summary stats
print("="*60)
print("GEMINI SENTIMENT CORRELATION SUMMARY")
print("="*60)
print(f"\nMean price change by sentiment:")
for s in ['positive','neutral','negative']:
    m = df[df['sentiment']==s]['percent_daily_price_change'].mean()*100
    print(f"  {s:10}: {m:+.3f}%")
print(f"\nOverall direction accuracy (pos/neg only): {df[df['sentiment']!='neutral']['correct'].mean()*100:.1f}%")
print(f"High confidence (>0.8) accuracy: {df[(df['confidence']>0.8) & (df['sentiment']!='neutral')]['correct'].mean()*100:.1f}%")